# 1. Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import os
import numpy as np
from torchvision import datasets, models, transforms

from torchvision.models import resnet18, ResNet18_Weights
import torch.optim as optim


import random
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split


from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedGroupKFold


from codes.utils import stratified_train_test_group_kfold
from codes.utils import model_test_classification
from codes.utils import accuracy_classification
from codes.utils import cnn_reg_cross_val_final_test

from codes.regression_codes import cnn_regression_gridsearch


torch.backends.cudnn.deterministic = True
random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)
np.random.seed(1)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

# 2. Data

In [3]:
with open("Data/MoS2_Analysis_Processed_Data2", "rb") as fp:   # Unpickling
  MoS2_data = pickle.load(fp)

df = pd.DataFrame(MoS2_data)


T_target = np.array(list(df['T']))
sampleId = np.array(list(df['sampleId']))
data_image = np.array([np.array(item) for item in df['image']])

#T_target = np.array(T_target).reshape(len(T_target),1)
print(len(T_target))

Data_CNN = data_image
print(Data_CNN.shape)
Data_CNN_rgb = np.repeat(Data_CNN[..., np.newaxis], 3, -1)
print(Data_CNN_rgb.shape)
Data_CNN_rgb = Data_CNN_rgb.transpose(0, 3, 1, 2)/255
print(Data_CNN_rgb.shape)


X = Data_CNN_rgb
Y = np.array(T_target)
#Y = Y.flatten()
groups = np.array(sampleId)

train_val_groups, train_val_X, train_val_Y, test_X, test_Y = stratified_train_test_group_kfold(X, Y, groups, n_splits=10, test_fold=0)


#Y = Y.flatten()



print(train_val_X.shape)
print(train_val_Y.shape)
print(test_X.shape)
print(test_Y.shape)




262
(262, 224, 224)
(262, 224, 224, 3)
(262, 3, 224, 224)
<class 'generator'>
(235, 3, 224, 224)
(235,)
(27, 3, 224, 224)
(27,)


# 3. Runing Cross-Validation

In [3]:
#!mkdir -p 'Models/regression/ImageNet/aug3/CNN/'

In [4]:
# the paramters below were optimized
# only the optimum parameters for the production model are used here


Learning_rate = [1e-4] #[1e-2, 5e-3, 1e-3, 5e-4, 1e-4, 1e-5, 5e-6] 
Drop_out = [0.0]#[0.2, 0.35, 0.5, 0.65, 0.75]
Batch_size = [32] #[24, 32, 100]

def cross_10_folds_cnn(train_val_X, train_val_Y):

    best_train = []
    best_val = []
    best_variables = []
    best_performance_record = []
    for fold in range(10):
        
        model_path = f'regression/ImageNet/aug3/CNN/{fold}_model.pth'
        group, train_X, train_Y, val_X, val_Y = stratified_train_test_group_kfold(train_val_X, train_val_Y, train_val_groups, n_splits=10, test_fold=fold)
        #train_X, val_X, train_Y, val_Y = train_test_split(train_val_X, train_val_Y, test_size=0.1,stratify=train_val_Y,random_state=fold+30)#42
        best_train_acc, best_val_acc,  best_record, hyper=cnn_regression_gridsearch(train_X, train_Y, val_X, val_Y,Learning_rate, Drop_out, Batch_size,fold,model_path, epochs=244)
        #best_train_acc, best_val_acc, trainf1, valf1, best_perf_record, hyper=cnn_class_train_fn(train_X, train_Y, val_X, val_Y, Learning_rate, Batch_size, fold)

        best_train.append(best_train_acc)
        best_val.append(best_val_acc)

        best_variables.append(hyper)
        best_performance_record.append(best_record)

        print(f'fold: {fold} done!')
    return best_train, best_val, best_variables, best_performance_record


best_train, best_val,best_variables, best_performance_record = cross_10_folds_cnn(train_val_X, train_val_Y)



<class 'generator'>
Epoch1: loss: 902741.3571 val_loss: 924948.1875
Epoch2: loss: 901501.2411 val_loss: 922409.6875
Epoch3: loss: 897749.5089 val_loss: 919828.3750
Epoch4: loss: 894991.8125 val_loss: 917175.3750
Epoch5: loss: 884204.1607 val_loss: 914411.0000
Epoch6: loss: 894648.4732 val_loss: 911501.6875
Epoch7: loss: 886219.0000 val_loss: 908563.7500
Epoch8: loss: 889054.0000 val_loss: 905330.6875
Epoch9: loss: 881061.6339 val_loss: 901987.6875
Epoch10: loss: 876312.4018 val_loss: 898392.3750
Epoch11: loss: 873760.3750 val_loss: 894321.1875
Epoch12: loss: 871572.8214 val_loss: 890247.0000
Epoch13: loss: 861148.5179 val_loss: 885855.1875
Epoch14: loss: 858218.7679 val_loss: 880753.4375
Epoch15: loss: 868009.3571 val_loss: 875232.8750
Epoch16: loss: 851738.8750 val_loss: 869875.6875
Epoch17: loss: 840057.6964 val_loss: 863691.4375
Epoch18: loss: 840616.7857 val_loss: 857422.1875
Epoch19: loss: 840904.2768 val_loss: 851184.4375
Epoch20: loss: 824692.7054 val_loss: 844936.9375
Epoch21: 

# 4. Test Models

In [5]:
def pretrained_model(drop_out):
    
    model = models.resnet18(weights=ResNet18_Weights.DEFAULT)

    model.fc = nn.Sequential(nn.ReLU(),
                            nn.Dropout(p=drop_out),
                                    nn.Linear(512, 100), #150, 50176
                                    nn.ReLU(),
                                    nn.Dropout(p=drop_out),
                                    nn.Linear(100,1)
                                     )
    model.to(device)
    return model

In [6]:

trained_model = pretrained_model(0.0)
root_path = f'regression/ImageNet/aug3/CNN/'

best_test, confusion_matrix_test, rmse_test_folds = cnn_reg_cross_val_final_test(trained_model, test_X, test_Y, 'test', root_path)


print('accuracy: ', best_test)
print('rmse: ', rmse_test_folds)
print(f'test_acc_mean: {np.mean(best_test) :.3f}, std: {np.std(best_test) :.3f}, rmse_mean: {np.mean(rmse_test_folds) :.3f}, rmse_std: {np.std(rmse_test_folds) :.3f}')


cm_list = []
for cm in confusion_matrix_test:
    cm_list.append([list(item) for item in cm])
#print(cm_list)

print(len(cm_list))
print('avg cm: ', [list(item) for item in np.mean(cm_list, axis=0)])


Accuracy of the network on the 27 test images: 51.9 %
Accuracy of the network on the 27 test images: 51.9 %
Accuracy of the network on the 27 test images: 59.3 %
Accuracy of the network on the 27 test images: 55.6 %
Accuracy of the network on the 27 test images: 55.6 %
Accuracy of the network on the 27 test images: 51.9 %
Accuracy of the network on the 27 test images: 74.1 %
Accuracy of the network on the 27 test images: 55.6 %
Accuracy of the network on the 27 test images: 55.6 %
Accuracy of the network on the 27 test images: 63.0 %
accuracy:  [51.851851851851855, 51.851851851851855, 59.25925925925926, 55.55555555555556, 55.55555555555556, 51.851851851851855, 74.07407407407408, 55.55555555555556, 55.55555555555556, 62.96296296296296]
rmse:  [35.59432172256991, 44.65913340072338, 30.712265626477674, 33.289757747169155, 35.33531474053908, 37.56665841279402, 29.913804004397914, 42.99524295256067, 38.20395104738374, 33.39272611083555]
test_acc_mean: 57.407, std: 6.468, rmse_mean: 36.166, 